In [1]:
import autograd
import autograd.numpy as np
import autograd.scipy as sp
import paragami
import scipy as osp

In [2]:
np.random.seed(42)

num_obs = 1000

# True values of parameters
true_sigma = \
    np.eye(3) * np.diag(np.array([1, 2, 3])) + \
    np.random.random((3, 3)) * 0.1
true_sigma = 0.5 * (true_sigma + true_sigma.T)

true_mu = np.array([0, 1, 2])

# Data
x = np.random.multivariate_normal(
    mean=true_mu, cov=true_sigma, size=(num_obs, ))

# Original weights.
original_weights = np.ones(num_obs)

In [3]:
def regularizer(par_dict, lam):
    return lam * np.sum(par_dict['mu'] ** 2)

def get_normal_log_prob(x, sigma, mu):
    sigma_inv = np.linalg.inv(sigma)
    sigma_det_sign, sigma_log_det = np.linalg.slogdet(sigma)
    if sigma_det_sign <= 0:
        return np.full(float('inf'), x.shape[0])
    else:
        x_centered = x - np.expand_dims(mu, axis=0)
        return -0.5 * (
            np.einsum('ni,ij,nj->n', x_centered, sigma_inv, x_centered) + \
            sigma_log_det)

def model_logpdf(par_dict, weights, x):
    # Note that the multivariate_normal.logpdf does not work when
    # the arguments are passed as keyword arguments.
    data_lpdf = \
        sp.stats.multivariate_normal.logpdf(
            x, par_dict['mu'], par_dict['sigma'])
    #data_lpdf = get_normal_log_prob(x, mu=par_dict['mu'], sigma=par_dict['sigma'])
    return np.sum(weights * data_lpdf)

def objective_fun(par_dict, weights, lam, x):
    return -model_logpdf(par_dict, weights, x) + regularizer(par_dict, lam)

norm_par = dict()
norm_par['mu'] = true_mu
norm_par['sigma'] = true_sigma

original_lam = 0

objective_fun(norm_par, original_weights, original_lam, x)

5149.567522214259

In [4]:
model_logpdf_params_mu_grad = autograd.grad(autograd.scipy.stats.multivariate_normal.logpdf, argnum=2)
print(model_logpdf_params_mu_grad(x[0, :], true_mu, true_sigma))

model_logpdf_params_mu_grad = autograd.grad(autograd.scipy.stats.multivariate_normal.logpdf, argnum=1)
print(model_logpdf_params_mu_grad(x[0, :], true_mu, true_sigma))

print(autograd.scipy.stats.multivariate_normal.logpdf(x[0, :], true_mu, true_sigma))
print(autograd.scipy.stats.multivariate_normal.logpdf(x[0, :], mean=true_mu, cov=true_sigma))



[[ 0.34540381  0.09413988  0.07870993]
 [ 0.09413988  0.15713758 -0.07289394]
 [ 0.07870993 -0.07289394  0.11167245]]
[ 0.52563825 -0.42833438 -0.32206316]
-4.15286246651169
-4.15286246651169


In [5]:
norm_param_pattern = paragami.PatternDict()
norm_param_pattern['sigma'] = paragami.PSDSymmetricMatrixPattern(size=3)
norm_param_pattern['mu'] = paragami.NumericArrayPattern(shape=(3, ))

def objective_par_only(par_dict):
    return objective_fun(par_dict, original_weights, original_lam, x)

objective_flat = paragami.FlattenFunctionInput(
    #lambda par_dict: objective(par_dict, original_weights, original_lam, x),
    objective_par_only,
    patterns=norm_param_pattern,
    free=True)

norm_par_flat = norm_param_pattern.flatten(norm_par, free=True)
assert(
    objective_flat(norm_par_flat) == \
    objective_fun(norm_par, original_weights, original_lam, x))

In [39]:
from autograd.test_util import check_grads

num_obs = 1000

# True values of parameters
true_sigma = \
    np.eye(3) * np.diag(np.array([1, 2, 3])) + \
    np.random.random((3, 3)) * 0.1
true_sigma = 0.5 * (true_sigma + true_sigma.T)

true_mu = np.array([0, 1, 2])

# Data
x = np.random.multivariate_normal(
    mean=true_mu, cov=true_sigma, size=(num_obs, ))

# Succeeds
check_grads(autograd.scipy.stats.multivariate_normal.logpdf, modes=['rev'])(x, true_mu, true_sigma)

# Fails
try:
    def logpdf_mu(mu):
        return np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, mu, true_sigma))
    print(logpdf_mu(true_mu))
    check_grads(logpdf_mu, modes=['rev'])(true_mu)
except AssertionError as err:
    print('Fails:\ntest_util.py 31:\tassert vspace(vjp_y) == x_vs')

try:
    def logpdf_mu(mu):
        return np.sum(autograd.scipy.stats.multivariate_normal.logpdf(x, mean=mu, cov=true_sigma))
    print(logpdf_mu(true_mu))
    check_grads(logpdf_mu, modes=['rev'])(true_mu)
except TypeError as err:
    print('Fails: ' + str(err))




-5149.567522214259
Fails:
test_util.py 31:	assert vspace(vjp_y) == x_vs
-5149.567522214259
Fails: Can't differentiate w.r.t. type <class 'numpy.int64'>


In [ ]:

from autograd.test_util import check_grads

objective_wrapper = paragami.OptimizationObjective(objective_flat)
init_param = np.ones(norm_param_pattern.flat_length(free=True))

check_grads(objective_flat, modes=['rev'])(init_param)
objective_wrapper.set_print_every(0)
check_grads(objective_wrapper.f, modes=['rev'])(init_param)


objective_wrapper.set_print_every(1)
objective_wrapper.reset()
mle_opt = osp.optimize.minimize(
    method='trust-ncg',
    x0=init_param,
    fun=objective_wrapper.f,
    jac=objective_wrapper.grad,
    hess=objective_wrapper.hessian,
    options={'gtol': 1e-12, 'disp': False})
print(mle_opt)

In [ ]:
print(mle_opt.x)
norm_par_opt = norm_param_pattern.fold(mle_opt.x, free=True)
print(norm_par_opt)